**Dependencies & Imports**

In [ ]:
!pip install pandas_market_calendars ripser persim

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import MDS
import statsmodels.api as sm
from scipy.cluster.hierarchy import linkage, dendrogram
import networkx as nx
from matplotlib.colors import Normalize
from scipy.spatial.distance import squareform
from tqdm import tqdm
import yfinance as yf
import pandas_datareader.data as web
import pandas_market_calendars as mcal
from datetime import datetime
import requests
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.cm as cm
from ripser import ripser
import networkx as nx
from numpy.polynomial.polynomial import Polynomial
from persim import plot_diagrams
from persim import PersLandscapeExact
from sklearn.cluster import AgglomerativeClustering
import scipy.cluster.hierarchy as sch

**Data Processing**

In [ ]:
components_df = pd.read_csv('/content/components.csv')

components_df['date'] = pd.to_datetime(components_df['date'])
components_df = components_df.set_index("date").sort_index()

nyse = mcal.get_calendar('NYSE')
schedule = nyse.schedule(start_date=components_df.index.min(), end_date=components_df.index.max())
trading_days = pd.DatetimeIndex(schedule.index.date)

components_df = components_df.reindex(trading_days)

components_df['tickers'] = components_df['tickers'].str.split(",")
components_df['tickers'] = components_df['tickers'].ffill()

components_df = components_df.reset_index().rename(columns={'index': 'date'})

components_df = components_df.set_index('date')

components_df

In [ ]:
prices_raw = pd.read_csv('/content/prices.csv')

prices_raw['date'] = pd.to_datetime(prices_raw['date'])

prices_df = prices_raw.pivot_table(index='date', columns='TICKER', values='PRC', aggfunc='first')

prices_df = prices_df.sort_index(axis=1)

prices_df = prices_df.abs()

mask = pd.DataFrame(False, index=prices_df.index, columns=prices_df.columns)

for date, row in components_df.iterrows():
    tickers = row['tickers']

    valid = list(set(tickers) & set(prices_df.columns))
    mask.loc[date, valid] = True

prices_df = prices_df.where(mask)

spx = yf.download('^GSPC', start='1996-01-01', end='2002-12-31')['Close']
prices_df.index = pd.to_datetime(prices_df.index)
spx.index = pd.to_datetime(spx.index)
prices_df = prices_df.join(spx, how='left')

prices_df

In [ ]:
spx_log_returns = np.log(spx / spx.shift(1)).dropna()
component_log_returns = np.log(prices_df / prices_df.shift(1))

component_log_returns = component_log_returns.loc[spx_log_returns.index]
component_log_returns = component_log_returns.fillna(0)

X = component_log_returns
y = spx_log_returns

model_no_alpha = sm.OLS(y, X).fit()
print(model_no_alpha.summary())

predicted = model_no_alpha.predict(X).to_numpy().flatten()
y_flat = y.to_numpy().flatten()

comparison = pd.DataFrame({
    'Actual': y_flat,
    'Synthetic': predicted
})

ax = comparison.cumsum().plot(title='Cumulative S&P 500 Log Return: Actual vs. Synthetic')
ax.set_xlabel("Day")
ax.set_ylabel("Cumulative Log Return")
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(y_flat, predicted, alpha=0.5)
plt.xlabel("Actual SPX Log Return")
plt.ylabel("Synthetic SPX Log Return")
plt.title("Actual vs Synthetic SPX Log Returns")
plt.plot([y_flat.min(), y_flat.max()], [y_flat.min(), y_flat.max()], color='red', lw=2)
plt.show()

**Correlation & Distance Matrices**

In [ ]:
window = 126
rolling_corrs_dict = {}

for t in range(window-1, len(prices_df)):
    window_data = prices_df.iloc[t-window+1:t+1]
    rolling_corrs_dict[prices_df.index[t]] = window_data.corr()

example_date = prices_df.index[window-1]
print(rolling_corrs_dict[example_date])

In [ ]:
corr_matrix = prices_df.corr()
distance_matrix = np.sqrt(2 * (1 - corr_matrix))
print(distance_matrix)

**Traditional Data Exploration**

In [ ]:
daily_stats = prices_df.describe()
print(daily_stats)

log_returns = np.log(prices_df / prices_df.shift(1))
daily_volatility = log_returns.std(axis=1)

daily_volatility.plot(title="Daily Volatility of Components")
plt.xlabel("Date")
plt.ylabel("Volatility")
plt.show()

In [ ]:
components_count = components_df['tickers'].apply(len)
components_count.plot(title="Number of Components in S&P 500 Over Time")

In [ ]:
window = 22

rolling_mean = prices_df.rolling(window).mean()
rolling_vol = prices_df.rolling(window).std()

rolling_vol.mean(axis=1).plot(title="Average Rolling Volatility of Components")

In [ ]:
distance_matrix = np.sqrt(2 * (1 - corr_matrix))

avg_distance = distance_matrix.mean().mean()
print(f"Average distance: {avg_distance}")

In [ ]:
log_returns_filled = np.log(prices_df / prices_df.shift(1)).fillna(0)
pca = PCA(n_components=919)
pca.fit(log_returns_filled)
explained_variance = pca.explained_variance_ratio_
print("Explained variance by all components:", explained_variance)

In [ ]:
window = 60
rolling_pc1_var = []

for t in range(window, len(log_returns_filled)):
    window_data = log_returns_filled.iloc[t-window:t]
    pca = PCA(n_components=1)
    pca.fit(window_data)
    rolling_pc1_var.append(pca.explained_variance_ratio_[0])

pd.Series(rolling_pc1_var, index=log_returns_filled.index[window:]).plot(title="Rolling Variance Explained by PC1")
plt.show()

In [ ]:
log_returns_filled = np.log(prices_df / prices_df.shift(1)).fillna(0)
pca = PCA(n_components=2)
pca_result = pca.fit_transform(log_returns_filled.T)

inertia = []
k_values = range(2, 15)
for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(pca_result)
    inertia.append(kmeans.inertia_)

plt.figure(figsize=(10,5))
plt.plot(k_values, inertia, marker='o')
plt.xlabel("Number of clusters k")
plt.ylabel("Inertia (Sum of Squared Distances)")
plt.title("Elbow Method for Optimal k")
plt.show()

sil_scores = []
for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42)
    labels = kmeans.fit_predict(pca_result)
    score = silhouette_score(pca_result, labels)
    sil_scores.append(score)

plt.figure(figsize=(10,5))
plt.plot(k_values, sil_scores, marker='o')
plt.xlabel("Number of clusters k")
plt.ylabel("Silhouette Score")
plt.title("Silhouette Scores for Different k")
plt.show()

In [ ]:
log_returns_filled = np.log(prices_df / prices_df.shift(1)).fillna(0)

pca = PCA(n_components=919)
pca_result = pca.fit_transform(log_returns_filled.T)

k = 2
kmeans = KMeans(n_clusters=k, random_state=42)
clusters = kmeans.fit_predict(pca_result)

plt.figure(figsize=(10,8))
for cluster_id in range(k):
    cluster_points = pca_result[clusters == cluster_id]
    plt.scatter(cluster_points[:,0], cluster_points[:,1], label=f"Cluster {cluster_id+1}", alpha=0.7)

for i, ticker in enumerate(log_returns_filled.columns):
    plt.text(pca_result[i,0], pca_result[i,1], ticker, fontsize=8)

plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title("PCA Clustering of S&P 500 Components")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
log_returns_filled = np.log(prices_df / prices_df.shift(1)).fillna(0)
pca = PCA(n_components=919)
pca_result = pca.fit_transform(log_returns_filled.T)

k = 2
kmeans = KMeans(n_clusters=k, random_state=42)
clusters = kmeans.fit_predict(pca_result)

cluster_df = pd.DataFrame({
    'Ticker': log_returns_filled.columns,
    'Cluster': clusters
})

stats_list = []
for cluster_id in range(k):
    tickers_in_cluster = cluster_df[cluster_df['Cluster'] == cluster_id]['Ticker']
    cluster_returns = log_returns_filled[tickers_in_cluster]

    mean_return = cluster_returns.mean(axis=1).mean()
    volatility = cluster_returns.std(axis=1).mean()
    avg_correlation = cluster_returns.corr().mean().mean()

    stats_list.append({
        'Cluster': cluster_id,
        'Num_Tickers': len(tickers_in_cluster),
        'Mean_Return': mean_return,
        'Volatility': volatility,
        'Avg_Correlation': avg_correlation
    })

cluster_stats = pd.DataFrame(stats_list)
print(cluster_stats)

In [ ]:
selected_date = '2000-03-10'
if selected_date in prices_df.index:
    corr_matrix_date = prices_df.loc[:selected_date].tail(window).corr()
    plt.figure(figsize=(10,8))
    sns.heatmap(corr_matrix_date, cmap='coolwarm', center=0)
    plt.title(f"Correlation Matrix around {selected_date}")
    plt.show()

In [ ]:
window = 126

avg_corr = log_returns.rolling(window).corr()
avg_corr = avg_corr.groupby(level=0).mean()
avg_vol = log_returns.rolling(window).std().mean(axis=1)

plt.figure(figsize=(10,6))
plt.plot(avg_vol.index, avg_vol, label="Average Volatility")
plt.plot(avg_corr.index, avg_corr, label="Average Correlation")
plt.title("Volatility vs Correlation Over Time")
plt.xlabel("Date")
plt.ylabel("Value")
plt.show()

**VC Complexes & Betti Numbers**

In [ ]:
window_size = 5
maxdim = 2
betti_results = []

for t in range(window_size - 1, len(prices_df)):
    window_data = prices_df.iloc[t-window_size+1:t+1].fillna(0)
    variances = window_data.var()
    sorted_stocks = variances.sort_values(ascending=False)

    cumsum_var = sorted_stocks.cumsum() / sorted_stocks.sum()
    alpha = 0.85
    top_stocks = cumsum_var[cumsum_var <= alpha].index

    window_data = window_data[top_stocks]

    if window_data.shape[1] < 2:
        continue

    print(f"Processing window ending on {prices_df.index[t]} with {window_data.shape[1]} tickers")

    corr_matrix = window_data.corr()
    dist_matrix = np.sqrt(np.clip(2 * (1 - corr_matrix), 0, 2))

    try:
        diagrams = ripser(dist_matrix.values, distance_matrix=True, maxdim=maxdim)['dgms']
        betti_0 = len(diagrams[0][~np.isinf(diagrams[0][:,1])])
        betti_1 = len(diagrams[1][~np.isinf(diagrams[1][:,1])])
    except Exception as e:
        print(f"Ripser failed at {prices_df.index[t]}: {e}")
        betti_0 = betti_1 = np.nan

    betti_results.append({
        'date': prices_df.index[t],
        'betti_0': betti_0,
        'betti_1': betti_1,
        'num_top_stocks': len(top_stocks)
    })

betti_df = pd.DataFrame(betti_results).set_index('date')
print(betti_df.head())

In [ ]:
window_data = prices_df.fillna(0)

corr_matrix = window_data.corr()
dist_matrix = np.sqrt(np.clip(2 * (1 - corr_matrix), 0, 2))

D = dist_matrix.values
nodes = list(dist_matrix.index)

thresholds = np.linspace(0, np.nanmax(D), 10)

for eps in thresholds:
    G = nx.Graph()
    G.add_nodes_from(nodes)

    for i in range(len(nodes)):
        for j in range(i+1, len(nodes)):
            if D[i, j] <= eps:
                G.add_edge(nodes[i], nodes[j])

    plt.figure(figsize=(7,7))
    nx.draw(
        G,
        with_labels=False,
        node_size=80,
        node_color='skyblue',
        edge_color='gray',
        linewidths=1.0,
        edgecolors='black'
    )
    plt.title(f"Vietoris–Rips 1-skeleton (ε = {eps:.2f})")
    plt.show()

In [ ]:
dates_num = np.arange(len(betti_df))
betti0 = betti_df['betti_0'].values
betti1 = betti_df['betti_1'].values

p0 = Polynomial.fit(dates_num, betti0, 10)
p1 = Polynomial.fit(dates_num, betti1, 10)

plt.figure(figsize=(12,6))
plt.plot(betti_df.index, betti0, alpha=0.5, label='Betti-0')
plt.plot(betti_df.index, betti1, alpha=0.5, label='Betti-1')
plt.plot(betti_df.index, p0(dates_num), label='Betti-0 Trend (Poly deg 3)', color='blue', lw=2)
plt.plot(betti_df.index, p1(dates_num), label='Betti-1 Trend (Poly deg 3)', color='orange', lw=2)
plt.title("Betti Numbers Over Time with Polynomial Trend")
plt.xlabel("Date")
plt.ylabel("Count")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(betti_df.index, betti_df['betti_0'] / betti_df['num_top_stocks'], label='Normalized Betti-0')
plt.plot(betti_df.index, betti_df['betti_1'] / betti_df['num_top_stocks'], label='Normalized Betti-1')
plt.title("Normalized Betti Numbers Over Time")
plt.xlabel("Date")
plt.ylabel("Normalized Count")
plt.legend()
plt.grid(True)
plt.show()

**Persistence Diagrams & Landscape**

In [ ]:
window_data = prices_df.fillna(0)

variances = window_data.var()
sorted_stocks = variances.sort_values(ascending=False)
cumsum_var = sorted_stocks.cumsum() / sorted_stocks.sum()

alpha = 0.85
top_stocks = cumsum_var[cumsum_var <= alpha].index

window_data = window_data[top_stocks]

corr_matrix = window_data.corr()
dist_matrix = np.sqrt(np.clip(2 * (1 - corr_matrix), 0, 2))

diagrams = ripser(dist_matrix.values, distance_matrix=True, maxdim=2)['dgms']

plot_diagrams(diagrams, show=False)
plt.title(f"Persistence Diagram of Stocks Explaining {int(alpha*100)}% Variance ({len(top_stocks)} stocks)")
plt.show()

In [ ]:
ple = PersLandscapeExact(dgms=diagrams, hom_deg=1)

plt.figure(figsize=(10,5))

for layer_idx, layer in enumerate(ple.critical_pairs):
    layer = np.array(layer)
    x = layer[:,0]
    y = layer[:,1]
    plt.plot(x, y, label=f"Layer {layer_idx+1}")

plt.title("Exact Persistence Landscape (H1)")
plt.xlabel("Filtration (epsilon)")
plt.ylabel("Landscape value")
plt.legend()
plt.show()

**Mapper Algorithm & Hierarchical Clustering**

In [ ]:
data = betti_df[['betti_0', 'betti_1']].values

scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)

mapper = km.KeplerMapper()

lens = PCA(n_components=2).fit_transform(data_scaled)

graph = mapper.map(
    lens,
    data_scaled,
    clusterer=km.cluster.DBSCAN(eps=0.5, min_samples=3),
    cover=km.Cover(n_cubes=10, perc_overlap=0.3)
)

mapper.visualize(
    graph,
    path_html="mapper_output.html",
    title="Mapper Algorithm on Dataset"
)

print("Mapper graph generated! Open 'mapper_output.html' to view the visualization.")

In [ ]:
data = betti_df[['betti_0', 'betti_1']].values

scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)

plt.figure(figsize=(10, 5))
dendrogram = sch.dendrogram(sch.linkage(data_scaled, method='ward'))
plt.title("Hierarchical Clustering Dendrogram")
plt.xlabel("Samples")
plt.ylabel("Euclidean distance")
plt.show()

n_clusters = 3
hc = AgglomerativeClustering(n_clusters=n_clusters, affinity='euclidean', linkage='ward')
labels = hc.fit_predict(data_scaled)

print("Cluster labels:", labels)